In [3]:
!pip install gradio plotly pandas numpy


In [4]:
import gradio as gr
import plotly.graph_objects as go
import pandas as pd
import numpy as np

In [13]:
QUESTIONS = [
    {"id": 1, "text": "Do you prioritize individual liberty over collective security?", "axis": "social", "direction": -1},
    {"id": 2, "text": "Do you believe that a strong judicial system and law enforcement are essential to maintaining social order?", "axis": "social", "direction": 1},
    {"id": 3, "text": "Do you affirm that everyone has the right to live freely as long as they do not infringe upon the rights of others?", "axis": "social", "direction": -1},
    {"id": 4, "text": "Do you value the preservation of cultural traditions and societal norms as part of our collective identity?", "axis": "social", "direction": 1},
    {"id": 5, "text": "Do you find government surveillance acceptable if it serves to protect society from crime?", "axis": "social", "direction": 1},
    {"id": 6, "text": "Do you believe civil liberties must be preserved even during times of crisis?", "axis": "social", "direction": -1},
    {"id": 7, "text": "Do you think that respect for authorities is necessary to maintain stability, regardless of differing opinions?", "axis": "social", "direction": 1},
    {"id": 8, "text": "Do you consider peaceful protest a guaranteed right that must be protected?", "axis": "social", "direction": -1},
    {"id": 9, "text": "Should the state promote moral values even if it limits some personal freedoms?", "axis": "social", "direction": 1},
    {"id": 10, "text": "Is it important for society to enforce conformity to shared cultural norms?", "axis": "social", "direction": 1},
    {"id": 11, "text": "Do you believe that minimizing government regulation of private businesses fosters entrepreneurial spirit and economic freedom?", "axis": "economic", "direction": 1},
    {"id": 12, "text": "Do you think providing universal healthcare is a fundamental responsibility of the government?", "axis": "economic", "direction": -1},
    {"id": 13, "text": "Do you agree that imposing fair taxes on the wealthy is necessary to achieve social equity?", "axis": "economic", "direction": -1},
    {"id": 14, "text": "Do you hold that free markets are the best means to achieve economic prosperity compared to government intervention?", "axis": "economic", "direction": 1},
    {"id": 15, "text": "Do you believe that workers have a legitimate right to form unions to protect their interests?", "axis": "economic", "direction": -1},
    {"id": 16, "text": "Do you accept that profit maximization should be the primary driver of corporate decisions?", "axis": "economic", "direction": 1},
    {"id": 17, "text": "Do you support the idea of a guaranteed basic income provided by the government to ensure economic dignity for all individuals?", "axis": "economic", "direction": -1},
    {"id": 18, "text": "Do you consider the protection of private property rights a cornerstone of free societies?", "axis": "economic", "direction": 1},
    {"id": 19, "text": "Should the government intervene to reduce income inequality?", "axis": "economic", "direction": -1},
    {"id": 20, "text": "Is economic growth best achieved by encouraging competition and reducing state control?", "axis": "economic", "direction": 1},
]


In [14]:


def calculate_political_position(*answers):
    """Calculate political position from quiz answers"""
    economic_score = 0
    social_score = 0
    economic_count = 0
    social_count = 0

    for i, question in enumerate(QUESTIONS):
        answer_value = answers[i]
        weighted_score = answer_value * question["direction"]

        if question["axis"] == "economic":
            economic_score += weighted_score
            economic_count += 1
        else:
            social_score += weighted_score
            social_count += 1

    if economic_count > 0:
        economic_score = (economic_score / economic_count) * 2.5
    if social_count > 0:
        social_score = (social_score / social_count) * 2.5

    if economic_score > 0 and social_score > 0:
        quadrant = "Authoritarian Right"
    elif economic_score < 0 and social_score > 0:
        quadrant = "Authoritarian Left"
    elif economic_score < 0 and social_score < 0:
        quadrant = "Libertarian Left"
    else:
        quadrant = "Libertarian Right"

    fig = go.Figure()

    fig.add_shape(type="rect", x0=-10, y0=0, x1=0, y1=10,
                  fillcolor="lightcoral", opacity=0.3, line_width=0)
    fig.add_shape(type="rect", x0=0, y0=0, x1=10, y1=10,
                  fillcolor="lightblue", opacity=0.3, line_width=0)
    fig.add_shape(type="rect", x0=-10, y0=-10, x1=0, y1=0,
                  fillcolor="lightgreen", opacity=0.3, line_width=0)
    fig.add_shape(type="rect", x0=0, y0=-10, x1=10, y1=0,
                  fillcolor="lightyellow", opacity=0.3, line_width=0)

    for i in range(-10, 11, 2):
        fig.add_hline(y=i, line_dash="dot", line_color="gray", opacity=0.5)
        fig.add_vline(x=i, line_dash="dot", line_color="gray", opacity=0.5)

    fig.add_hline(y=0, line_color="black", line_width=2)
    fig.add_vline(x=0, line_color="black", line_width=2)

    fig.add_trace(go.Scatter(
        x=[economic_score], y=[social_score],
        mode='markers',
        marker=dict(size=20, color='red', symbol='circle'),
        name='Your Position',
        hovertemplate='Economic: %{x:.1f}<br>Social: %{y:.1f}<extra></extra>'
    ))

    fig.add_annotation(x=-5, y=5, text="Authoritarian<br>Left", showarrow=False, font_size=14)
    fig.add_annotation(x=5, y=5, text="Authoritarian<br>Right", showarrow=False, font_size=14)
    fig.add_annotation(x=-5, y=-5, text="Libertarian<br>Left", showarrow=False, font_size=14)
    fig.add_annotation(x=5, y=-5, text="Libertarian<br>Right", showarrow=False, font_size=14)

    fig.update_layout(
        title="Your Political Compass Result",
        xaxis_title="Economic Axis (Left ← → Right)",
        yaxis_title="Social Axis (Libertarian ← → Authoritarian)",
        xaxis=dict(range=[-10, 10], zeroline=False),
        yaxis=dict(range=[-10, 10], zeroline=False),
        width=700,
        height=700,
        showlegend=False
    )

    results_text = f"""
    ## Your Political Position: {quadrant}

    **Economic Score:** {economic_score:.1f} (Negative = Left, Positive = Right)
    **Social Score:** {social_score:.1f} (Negative = Libertarian, Positive = Authoritarian)

    ### Interpretation:
    """

    if quadrant == "Authoritarian Right":
        results_text += "You tend to favor free-market economics while supporting strong government authority in social matters."
    elif quadrant == "Authoritarian Left":
        results_text += "You tend to favor government intervention in economics while supporting strong government authority in social matters."
    elif quadrant == "Libertarian Left":
        results_text += "You tend to favor government intervention in economics while supporting individual freedom in social matters."
    else:
        results_text += "You tend to favor free-market economics while supporting individual freedom in social matters."

    return fig, results_text



In [15]:
def create_quiz_interface():
    inputs = []

    for i, question in enumerate(QUESTIONS):
        inputs.append(
            gr.Slider(
                minimum=1,
                maximum=5,
                value=3,
                step=1,
                label=f"Q{i+1}: {question['text']}",
                info="1=Strongly Disagree, 2=Disagree, 3=Neutral, 4=Agree, 5=Strongly Agree"
            )
        )

    outputs = [
        gr.Plot(label="Political Compass"),
        gr.Markdown(label="Results")
    ]

    iface = gr.Interface(
        fn=calculate_political_position,
        inputs=inputs,
        outputs=outputs,
        title="🗳️ Political Compass Quiz",
        description="Answer all questions to discover your position on the political compass!",
        theme=gr.themes.Soft(),
        allow_flagging="never"
    )

    return iface

In [16]:
quiz_app = create_quiz_interface()
quiz_app.launch(share=True, debug=True)

print("🎉 Your Political Compass Quiz is now live!")
print("📋 The public URL will appear above - you can share it with anyone!")
print("⏰ Keep this cell running to keep the quiz active")


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning:

The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.



Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://72ef77de98fc1a17f5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://72ef77de98fc1a17f5.gradio.live
🎉 Your Political Compass Quiz is now live!
📋 The public URL will appear above - you can share it with anyone!
⏰ Keep this cell running to keep the quiz active
